## Face Dataset Collection

In [1]:
import cv2
import os

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create dataset directory if not exists
DATASET_DIR = './dataset'
os.makedirs(DATASET_DIR, exist_ok=True)

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

while True:
    person_name = input("Enter the person's name (or type 'done' to finish): ").strip()
    if person_name.lower() == 'done':
        print("Dataset collection complete!")
        break

    person_dir = os.path.join(DATASET_DIR, person_name)
    if os.path.exists(person_dir):
        print(f"Images for {person_name} already exist. Skipping...")
        continue
    os.makedirs(person_dir, exist_ok=True)

    print(f"Collecting images for {person_name}. Press 'q' to stop.")
    count = 0

    while count < 100:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame from webcam.")
            break

        # Convert to grayscale for better face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            padding = 10
            x1, y1 = max(0, x - padding), max(0, y - padding)
            x2, y2 = min(frame.shape[1], x + w + padding), min(frame.shape[0], y + h + padding)

            # Draw rectangle around face
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Save the cropped face
            face_roi = frame[y1:y2, x1:x2]
            img_path = os.path.join(person_dir, f"{person_name}_{count}.jpg")
            cv2.imwrite(img_path, face_roi)
            count += 1
            print(f"Captured {count}/100 images for {person_name}")

            if count >= 100:
                break

        # Display the frame
        cv2.imshow("Face Capture - Press 'q' to Quit", frame)

        # Quit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Captured 1/100 images for ali
Captured 2/100 images for ali
Captured 3/100 images for ali
Captured 4/100 images for ali
Captured 5/100 images for ali
Captured 6/100 images for ali
Captured 7/100 images for ali
Captured 8/100 images for ali
Captured 9/100 images for ali
Captured 10/100 images for ali
Captured 11/100 images for ali
Captured 12/100 images for ali
Captured 13/100 images for ali
Captured 14/100 images for ali
Captured 15/100 images for ali
Captured 16/100 images for ali
Captured 17/100 images for ali
Captured 18/100 images for ali
Captured 19/100 images for ali
Captured 20/100 images for ali
Captured 21/100 images for ali
Captured 22/100 images for ali
Captured 23/100 images for ali
Captured 24/100 images for ali
Captured 25/100 images for ali
Captured 26/100 images for ali
Captured 27/100 images for ali
Captured 28/100 images for ali
Captured 29/100 images for ali
Captured 30/100 images for ali
Captured 31/100 images for ali
Captured 32/100 images for ali
Captured 33/100 i

## Data Preprocessing

In [2]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

IMG_SIZE = 128  # Image size for input
DATA_DIR = './dataset'

# Get class labels from dataset folder names
persons = [name for name in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, name))]
labels = {name: i for i, name in enumerate(persons)}

# Load dataset images
X, y = [], []
for person in persons:
    folder = os.path.join(DATA_DIR, person)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        try:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img)
                y.append(labels[person])
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

X = np.array(X) / 255.0  # Normalize images
y = np.array(y)

if len(X) == 0:
    print("No valid data found. Exiting...")
    exit()

# Split dataset into training & validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=len(persons))
y_val = to_categorical(y_val, num_classes=len(persons))

print("Dataset preprocessed successfully!")


Dataset preprocessed successfully!


## Model Training with Transfer Learning

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Load the preprocessed data
IMG_SIZE = 128  # Image size used in preprocessing
NUM_CLASSES = len(persons)  # Number of classes (persons)

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16)

# Save the model
model.save('attendance_resnet_model.h5')
print("ResNet50 model trained and saved successfully!")


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 277ms/step - accuracy: 0.5094 - loss: 0.8881 - val_accuracy: 0.4750 - val_loss: 0.7032
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - accuracy: 0.4640 - loss: 0.8226 - val_accuracy: 0.5250 - val_loss: 0.6788
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 163ms/step - accuracy: 0.4626 - loss: 0.7780 - val_accuracy: 0.4750 - val_loss: 0.6625
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.5268 - loss: 0.7452 - val_accuracy: 0.9000 - val_loss: 0.6313
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 163ms/step - accuracy: 0.4927 - loss: 0.7120 - val_accuracy: 0.5250 - val_loss: 0.6290
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 0.6078 - loss: 0.6891 - val_accuracy: 0.8750 - val_loss: 0.6015
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.5699 - loss: 0.6794 - val_accuracy: 0.9000 - val_loss: 0.5915
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 170ms/step - accuracy: 0.6943 - loss: 0.6298 - val_accuracy: 0.

ResNet50 model trained and saved successfully!


## Real-Time Face Recognition

In [ ]:
from tensorflow.keras.models import load_model

# Load trained model
model = load_model('attendance_resnet_model.h5')

# Start webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        # Draw rectangle on detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Preprocess face for model
        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (IMG_SIZE, IMG_SIZE))
        face_roi = np.expand_dims(face_roi, axis=0) / 255.0

        # Predict person
        predictions = model.predict(face_roi)
        person_index = np.argmax(predictions)
        confidence = np.max(predictions)

        label = list(labels.keys())[person_index] if confidence > 0.7 else "Unidentified"

        # Display result
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow("Face Recognition - Press 'q' to Quit", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 996ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━